In [ ]:
# https://colah.github.io/posts/2015-08-Understanding-LSTMs/
# 이 lstm을 만든사람은 고졸이었음
# rnn: 순환신경망

# cnn: 입력 - 합성곱(필터들과 입력 사이에서 합성곱 연산 수행=피처맵이 됨)-풀링(일반적으로 맥스풀링)-합성곱-풀링..-평탄화-완전연결(fully connected)-출력
# rnn: 입력 - rnn(현재 입력했던 구간으로 돌아감) 즉 rnn신경망이 들어가있음 - 출력
# rnn은 순서가 있는 데이터들로 구성되는 것이 일반적임
# rnn은 가장 최근의 정보만 이용하려는 특성이 있어 이것이 장점이자 단점임
# 예를 들어 the clouds are in the sky 라는 문장에서 sky가 비어 있어 맞추려고 한다면 문장에서 다른 정보가 더 없더라도 예측이 가능함
# 하지만 문장이 더 길어지게 되면 rnn이 제대로 동작하지 않게 됨(장기 의존성 문제)

# 예시로
# h0=clouds, h1=are, h2=in, h3=the 
# x0=the (h0는 다시 입력으로 반복...)  이렇게 되어 있을 때 h4는 무엇일까? = 실제 정답은 sky임
# 근데 rnn은 주변에 있는 몇개의 단어로 예측하는 모델임

# 다른 예시로 I grew up in France… I speak fluent French 이런 문장이 있을 때
# French가 비어있고 이 단어를 예측하려고 한다면 rnn으로 예측이 안됨(rnn은 I speak fluent 이 문장으로 예측하려고 하기 때문)
# 즉 더 이전에 언급했던 내용도 정보로 가지고 있어야 하는데 단기적인 기억으로만 예측하려고 하므로 정확도가 떨어짐

# 다른 예시로 주식가격을 예측하는 데에도 최근 데이터만 가지고 예측하기엔 다소 무리가 있듯 rnn의 한계점이 명확함
# 이를 극복한 모델이 LSTM임(물론 lstm도 문제는 많음)

# rnn와 lstm의 차이점은 장기 의존성을 해결한 모델이 lstm임
# lstm에는 게이트라고 불리는 구조가 있어 정보를 선별적으로 전달하거나 차단하는 방법으로 장기 의존성 문제를 해결하려고 함
# 대학원 시험문제로 lstm, gru가 무엇인지 설명하시오. 등의 문제를 출제하기도 하므로 대학원에서는 익혀야함



https://ctkim.tistory.com/entry/LSTMLong-short-time-memory-%EA%B8%B0%EC%B4%88-%EC%9D%B4%ED%95%B4
위에서 개념이해하기
https://kr.mathworks.com/discovery/rnn.html rnn

https://github.com/gilbutITbook/080324/blob/master/ch17.ipynb 여기 참고

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding
from tensorflow.keras.utils import to_categorical
from numpy import array

In [9]:
# 케라스의 텍스트 전처리와 관련한 함수중 text_to_word_sequence 함수를 불러옵니다.
from tensorflow.keras.preprocessing.text import text_to_word_sequence

In [10]:
# 전처리할 텍스트를 정합니다.
text = '해보지 않으면 해낼 수 없다'

In [11]:
# 해당 텍스트를 토큰화합니다. 
result = text_to_word_sequence(text)
print("\n원문:\n", text)
print("\n토큰화:\n", result) # 토큰화는 띄어쓰기를 기준으로 이루어짐 (띄어쓰기 -> 형태소 단위)
# 근데 이럴라면 split 써도 되는 거 아님? => 그건 아님
# 토큰화 도구마다 다르다고 볼 수 있음. 
# 물론 여기서는 띄어쓰기를 기반으로 두게 된 것이기 때문에 언어적 특성을 반영한다고 보면 됨


원문:
 해보지 않으면 해낼 수 없다

토큰화:
 ['해보지', '않으면', '해낼', '수', '없다']


In [ ]:
# 토큰이 무엇인가? = 언어모델에서 자연어처리를 수행하는 처리 단위(단어, 문장, 문단, 글자,.. 단위는 모델에 따라 천차만별임)
# 자연어처리 도구 : konlpy(mecab, okt, kkma,...), nltk, keras, ..
# keras.preprocessing.text import text_to_word_sequence 에도 포함되어 있음
# 물론 파이토치에도 존재함
# 근데 konlpy를 쓰는 것이 좋긴함

# ex) 언어 모델링 과정
# 1) 텍스트 수집
# 2) 텍스트 전처리 & 토큰화(nltk, konlpy, ...)
# 3) 텍스트 인코딩
# 4) 텍스트 패딩
# 5) 모델링(LSTM, BERT, GPT ...)
# 6) 모델 학습
# 7) 모델 평가

## 단어 빈도수 세기

In [21]:
# 전처리하려는 세 개의 문장을 정합니다.
docs = ['먼저 텍스트의 각 단어를 나누어 토큰화 합니다.',
       '텍스트의 단어로 토큰화해야 딥러닝에서 인식됩니다.',
       '토큰화한 결과는 딥러닝에서 사용할 수 있습니다.',
       ]

In [22]:
# 토큰화 함수를 이용해 전처리 하는 과정입니다.
token = Tokenizer()             # 토큰화 함수 지정
token.fit_on_texts(docs)       # 토큰화 함수에 문장 적용
# 코퍼스(corpus): 특정 도메인 낱말 집합(일반용어사전x, 전문용어사전)
# 법률사전(피의자, 피해자, 가해자,...), 의료사전(수술, 환자, 감염병,..)

In [23]:
# 단어의 빈도수를 계산한 결과를 각 옵션에 맞추어 출력합니다.
# Tokenizer()의 word_counts 함수는 순서를 기억하는 OrderedDict 클래스를 사용합니다.
print("\n단어 카운트:\n", token.word_counts) 


단어 카운트:
 OrderedDict([('먼저', 1), ('텍스트의', 2), ('각', 1), ('단어를', 1), ('나누어', 1), ('토큰화', 1), ('합니다', 1), ('단어로', 1), ('토큰화해야', 1), ('딥러닝에서', 2), ('인식됩니다', 1), ('토큰화한', 1), ('결과는', 1), ('사용할', 1), ('수', 1), ('있습니다', 1)])


In [24]:
print("\n문장 카운트: ", token.document_count)


문장 카운트:  3


In [25]:
print("\n각 단어가 몇 개의 문장에 포함되어 있는가:\n", token.word_docs)


각 단어가 몇 개의 문장에 포함되어 있는가:
 defaultdict(<class 'int'>, {'합니다': 1, '텍스트의': 2, '나누어': 1, '단어를': 1, '토큰화': 1, '각': 1, '먼저': 1, '인식됩니다': 1, '단어로': 1, '토큰화해야': 1, '딥러닝에서': 2, '사용할': 1, '있습니다': 1, '결과는': 1, '수': 1, '토큰화한': 1})


In [26]:
print("\n각 단어에 매겨진 인덱스 값:\n",  token.word_index)


각 단어에 매겨진 인덱스 값:
 {'텍스트의': 1, '딥러닝에서': 2, '먼저': 3, '각': 4, '단어를': 5, '나누어': 6, '토큰화': 7, '합니다': 8, '단어로': 9, '토큰화해야': 10, '인식됩니다': 11, '토큰화한': 12, '결과는': 13, '사용할': 14, '수': 15, '있습니다': 16}


In [40]:
text="오랫동안 꿈꾸는 이는 그 꿈을 닮아간다 꿈꾸는 사람은 꿈을" # 코퍼스로 가정. 
# 테스트로  '꿈꾸는 사람은 꿈을' 이라는 문장도 덧붙여서 확인해도 좋음

In [41]:
token = Tokenizer()
token.fit_on_texts([text])
print(token.word_index)

{'꿈꾸는': 1, '꿈을': 2, '오랫동안': 3, '이는': 4, '그': 5, '닮아간다': 6, '사람은': 7}


In [43]:
sent1="나는 꿈을 꿨습니다 사람은 꿈을 꿉니다"
sent2="열이 많이 나고 기침이 나는데요. 어떻게 치료를 받아야 할까요?"
token.texts_to_sequences([sent1]) # text에서 '꿈꾸는 사람은 꿈을' 도 넣으면 [[2,7,2]]로 출력됨
# 나는x 꿈을2, 꿨습니다x 사람은7 꿈을2 꿉니다x  이런식으로 변환됨(즉 인코딩을 말하는 것임)
# 코퍼스 -> 전처리 -> 언어모델(lstm기반 감정분류기 만들었다면...)
# 테스트: 새로운문자(나는 꿈을 꿨습니다 사람은 꿈을 꿉니다) -> 인코딩([[2, 7, 2]]) -> 언어모델(lstm기반 감정분류기) -> 긍정/부정

[[2, 7, 2]]

In [44]:
# 챗봇을 만들려고 한다면...
# 대화를 나눈 텍스트 데이터가 필요. 이 데이터는 코퍼스라고도 부름

{'꿈꾸는': 1, '꿈을': 2, '오랫동안': 3, '이는': 4, '그': 5, '닮아간다': 6, '사람은': 7}

In [45]:
x=token.texts_to_sequences([sent1])
print(x)

[[2, 7, 2]]


In [47]:
text="오랫동안 꿈꾸는 이는 그 꿈을 닮아간다"

In [49]:
x = token.texts_to_sequences([text])
x

[[3, 1, 4, 5, 2, 6]]

In [50]:
#인덱스 수에 하나를 추가해서 원-핫 인코딩 배열 만들기
word_size = len(token.word_index) + 1 # 1을 더한 이유는 0번부터 시작하기 때문임. 쉽게 말하면 앞에 0을 추가하기 위해서 사용한 것임
x = to_categorical(x, num_classes=word_size)
print(x)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0.]]]


In [52]:
# 텍스트 리뷰 자료를 지정합니다.
docs = ["너무 재밌네요","최고예요","참 잘 만든 영화예요","추천하고 싶은 영화입니다","한번 더 보고싶네요","글쎄요","별로예요","생각보다 지루하네요","연기가 어색해요","재미없어요"]
# 위의 경우는 '영화 리뷰 코퍼스' 라고 할 수 있음. 여기에 없는 것은 코퍼스라고 할 수 없음
# 긍정 리뷰는 1, 부정 리뷰는 0으로 클래스를 지정합니다.
classes = array([1,1,1,1,1,0,0,0,0,0])

In [53]:
# 토큰화 
token = Tokenizer()
token.fit_on_texts(docs)
print(token.word_index) 
# 영화 리뷰 코퍼스에는 토큰 20개가 있음

{'너무': 1, '재밌네요': 2, '최고예요': 3, '참': 4, '잘': 5, '만든': 6, '영화예요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로예요': 15, '생각보다': 16, '지루하네요': 17, '연기가': 18, '어색해요': 19, '재미없어요': 20}


In [55]:
x = token.texts_to_sequences(docs)
print("\n리뷰 텍스트, 토큰화 결과:\n", x)
# 패딩: 모든 문장의 길이(토큰의 개수)를 동일하게 맞춰주는 작업
# 현재는 토큰의 최대 길이가 4이므로 4에 맞춰야 함


리뷰 텍스트, 토큰화 결과:
 [[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13], [14], [15], [16, 17], [18, 19], [20]]


In [56]:
# 패딩, 서로 다른 길이의 데이터를 4로 맞추어 줍니다.
padded_x = pad_sequences(x, 4)  
print("\n패딩 결과:\n", padded_x)


패딩 결과:
 [[ 0  0  1  2]
 [ 0  0  0  3]
 [ 4  5  6  7]
 [ 0  8  9 10]
 [ 0 11 12 13]
 [ 0  0  0 14]
 [ 0  0  0 15]
 [ 0  0 16 17]
 [ 0  0 18 19]
 [ 0  0  0 20]]


In [58]:
# 임베딩에 입력될 단어의 수를 지정합니다.
word_size = len(token.word_index) +1
word_size # 기존에는 1~20이었지만 1을 추가했으므로 0~20이라고 보면 됨

21

In [59]:
# 단어 임베딩을 포함하여 딥러닝 모델을 만들고 결과를 출력합니다.
model = Sequential()
model.add(Embedding(word_size, 8, input_length=4))
# dnn은 dense, cnn은 cov2d, lstm은 embedding을 사용함
model.add(Flatten())
model.add(Dense(1, activation='sigmoid')) # 긍정/부정 분류이므로 유닛수를 1로 설정
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 4, 8)              168       
                                                                 
 flatten_1 (Flatten)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________


In [64]:
# 21*8이 파라미터 계산값이며 중간에 껴있는 4는 토큰 최대길이를 의미함
# 평탄화는 파라미터가 아니지만 요약상 표기는 함
# 이후 dense에 들어가는데 32차원에서 결과가 1차원이므로 bias는 1로, 파라미터는 33이됨

168

In [ ]:
# https://cs231n.github.io/rnn/
# 정리하면 코퍼스를 가진 상태에서 토큰화
# 주소의 그림에서 hello를 그림으로 표기한 것이 있는데
# 일반적인 딥러닝 모델로는 다음 글자를 예측할 때 l다음 l이 나오는 건 이해해도, 
# 똑같이 l 다음 o가 나오는 것에 대해서는 이해를 할 수 없음

In [65]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(padded_x, classes, epochs=20)
print("\n Accuracy: %.4f" % (model.evaluate(padded_x, classes)[1]))

Epoch 1/20
1/1 [==============================] - 0s 418ms/step - loss: 0.6959 - accuracy: 0.4000
Epoch 2/20
1/1 [==============================] - 0s 8ms/step - loss: 0.6938 - accuracy: 0.4000
Epoch 3/20
1/1 [==============================] - 0s 8ms/step - loss: 0.6918 - accuracy: 0.4000
Epoch 4/20
1/1 [==============================] - 0s 12ms/step - loss: 0.6897 - accuracy: 0.5000
Epoch 5/20
1/1 [==============================] - 0s 8ms/step - loss: 0.6876 - accuracy: 0.5000
Epoch 6/20
1/1 [==============================] - 0s 9ms/step - loss: 0.6855 - accuracy: 0.7000
Epoch 7/20
1/1 [==============================] - 0s 10ms/step - loss: 0.6835 - accuracy: 0.8000
Epoch 8/20
1/1 [==============================] - 0s 11ms/step - loss: 0.6814 - accuracy: 0.8000
Epoch 9/20
1/1 [==============================] - 0s 10ms/step - loss: 0.6793 - accuracy: 0.8000
Epoch 10/20
1/1 [==============================] - 0s 7ms/step - loss: 0.6773 - accuracy: 0.9000
Epoch 11/20
1/1 [================

In [66]:
# 마지막은 100%의 정확도를 보였음(물론 학습에 따라 다른 결과를 보일 수 있음)

In [71]:
# 입력 데이터는 2차원 배열로 입력이 되어야함
# msg = array([[0,4,2,3]]) # 참 재밌네요 최고예요 - 긍정으로 출력(>0.5)
# msg = array([[0,0,0,17]]) # 지루하네요 - 부정으로 출력(<0.5)
msg = array([[0,0,4,17]]) # 참 지루하네요 - 부정으로 출력(<0.5)
model.predict(msg) # 인코딩하여 예측해야됨

# 질문사항 : 예측이 완전히 맞는 것은 거의 불가능함. 그러나 답은 맞출 수 있으므로 역전파는 사실상 무조건 진행된다고 볼 수 있음

array([[0.4784944]], dtype=float32)

In [ ]:
# 3시40분 디자인씽킹 설명
# 이후 회의실 생성하여 프로젝트 주제 관련 얘기하도록 시간 줄것임
# 5시쯤 어떤 것을 진행할 것인지 발표할 것임